# 1. Design a NIN network

In [ ]:
import numpy as np
# import pandas as pd
# import tensorflow as tf

In [ ]:
class ImplementNIN:
    def __init__(self, input, kernel, stride=1):
        self.input = input
        self.kernel = kernel
        self.stride = stride
        self.input_L = input.shape[0]
        self.input_W = input.shape[1]
        self.input_H = input.shape[2]
        self.kernel_L = kernel[0]
        self.kernel_H = kernel[1]
        output_L = self.input_L
        self.output = np.arange()
        pass
    
    def InitNetworkArchitecture(self):
        for r in 10:
            for c in 10:
                